In [68]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [49]:
# # Load trips (use only columns needed to save space)
# #usecols = [
#     "ride_id", "rideable_type", "started_at", "ended_at",
#     "start_station_name", "end_station_name",
#     "start_lat", "start_lng", "end_lat", "end_lng", "member_casual"
# ]
# trips = pd.read_csv("citibike_2022.csv", usecols=usecols, parse_dates=["started_at", "ended_at"])

# # Filter: first 6 months of 2022 (Jan–Jun)
# trips_6m = trips[trips["started_at"].dt.month <= 6]

# # Save reduced dataset
# trips_6m.to_csv("citibike_2022_6months.csv", index=False)
# print("Saved citibike_2022_6months.csv:", trips_6m.shape)


In [50]:
# # Load trips (use only columns needed to save space)
# usecols = [
#     "ride_id", "rideable_type", "started_at", "ended_at",
#     "start_station_name", "end_station_name",
#     "start_lat", "start_lng", "end_lat", "end_lng", "member_casual"
# ]
# trips = pd.read_csv("citibike_2022.csv", usecols=usecols, parse_dates=["started_at", "ended_at"])

# # Filter: first 6 months of 2022 (Jan–Jun)
# trips_3m = trips[trips["started_at"].dt.month <= 3]

# # Save reduced dataset
# trips_6m.to_csv("citibike_2022_3months.csv", index=False)
# print("Saved citibike_2022_3months.csv:", trips_3m.shape)

In [51]:
# # Load trips (use only columns needed to save space)
# usecols = [
#     "ride_id", "rideable_type", "started_at", "ended_at",
#     "start_station_name", "end_station_name",
#     "start_lat", "start_lng", "end_lat", "end_lng", "member_casual"
# ]
# trips = pd.read_csv("citibike_2022.csv", usecols=usecols, parse_dates=["started_at", "ended_at"])

# # Filter: first 6 months of 2022 (Jan–Jun)
# trips_2m = trips[trips["started_at"].dt.month <= 2]

# # Save reduced dataset
# trips_2m.to_csv("citibike_2022_2months.csv", index=False)
# print("Saved citibike_2022_2months.csv:", trips_2m.shape)

In [52]:
# Load trips (use only columns needed to save space)
usecols = [
    "ride_id", "rideable_type", "started_at", "ended_at",
    "start_station_name", "end_station_name",
    "start_lat", "start_lng", "end_lat", "end_lng", "member_casual"
]
trips = pd.read_csv("citibike_2022.csv", usecols=usecols, parse_dates=["started_at", "ended_at"])

# Filter: first 6 months of 2022 (Jan–Jun)
trips_1m = trips[trips["started_at"].dt.month <= 1]

# Save reduced dataset
trips_1m.to_csv("citibike_2022_1months.csv", index=False)
print("Saved citibike_2022_1months.csv:", trips_1m.shape)

Saved citibike_2022_1months.csv: (109, 11)


In [53]:
# Load weather
weather = pd.read_csv("laguardia_weather_2022.csv")

# Ensure datetime
weather["date"] = pd.to_datetime(weather["date"].astype(str).str[:10], errors="coerce")

# Filter: first month
weather_1m = weather[weather["date"].dt.month <= 1]

# Save reduced dataset
weather_1m.to_csv("laguardia_weather_2022_1months.csv", index=False)
print("Saved laguardia_weather_2022_1months.csv:", weather_1m.shape)

Saved laguardia_weather_2022_1months.csv: (93, 5)


In [54]:
# 1) Load trips sample (<25 MB)

trips = pd.read_csv("citibike_2022_1months.csv", parse_dates=["started_at"])


# 2) Build Top Stations (for bar)

top_stations = (
    trips.groupby("start_station_name", as_index=False)
         .size()
         .rename(columns={"size": "trip_count"})
         .sort_values("trip_count", ascending=False)
)
top_stations.to_csv("top_stations.csv", index=False)
print("✅ Saved top_stations.csv")


# 3) Load & prep LaGuardia weather (NOAA CSV)

# Expected columns (typical NOAA daily): date, datatype (TMAX/TMIN/PRCP), value
# - value usually in tenths (°C for temps, mm for precipitation)
w = pd.read_csv("laguardia_weather_2022_1months.csv")

# normalize column names
w.columns = [c.strip().lower() for c in w.columns]

# minimal column checks
if not {"date", "datatype", "value"} <= set(w.columns):
    raise KeyError("Weather CSV must have at least columns: date, datatype, value")

# ensure datetime
w["date"] = pd.to_datetime(w["date"].astype(str).str[:10], errors="coerce")

# pivot to wide: one row per day with TMAX/TMIN/PRCP columns
weather_daily = (
    w.pivot_table(index="date", columns="datatype", values="value", aggfunc="mean")
      .reset_index()
)

# If NOAA tenths-of-units, convert:
# - Temps (TMAX/TMIN) tenths of °C → °C (and optional °F)
# - PRCP tenths of mm → mm
for col in ["TMAX", "TMIN", "PRCP"]:
    if col in weather_daily.columns:
        # convert tenths to 1x units
        weather_daily[col] = weather_daily[col] / 10.0

# create a convenient average temperature column in °C (and optional °F)
if {"TMAX", "TMIN"} <= set(weather_daily.columns):
    weather_daily["TAVG_C"] = (weather_daily["TMAX"] + weather_daily["TMIN"]) / 2.0
elif "TMAX" in weather_daily.columns:
    weather_daily["TAVG_C"] = weather_daily["TMAX"]
elif "TMIN" in weather_daily.columns:
    weather_daily["TAVG_C"] = weather_daily["TMIN"]
else:
    weather_daily["TAVG_C"] = pd.NA

weather_daily["TAVG_F"] = weather_daily["TAVG_C"] * 9/5 + 32


# 4) Daily trips + merge WX

daily_trips = (
    trips.assign(date=trips["started_at"].dt.date)
         .groupby("date", as_index=False)["started_at"]
         .count()
         .rename(columns={"started_at": "trip_count"})
)
daily_trips["date"] = pd.to_datetime(daily_trips["date"])

# merge with weather on date
daily = daily_trips.merge(weather_daily, on="date", how="left")
daily.to_csv("daily_trips.csv", index=False)
print("Saved daily_trips.csv (with weather columns)")


# 5) Plotly – Top stations bar (notebook)

fig_bar = px.bar(
    top_stations.head(20),
    x="start_station_name",
    y="trip_count",
    title="Top 20 Most Popular Start Stations",
    color="trip_count",
    color_continuous_scale="Blues",
    labels={"start_station_name": "Station", "trip_count": "Trips"}
)
fig_bar.update_layout(xaxis_tickangle=45)
fig_bar.show()


# 6) Plotly – Dual-axis: Trips vs Temperature (TAVG_F)

# choose a temperature column to display; prefer TAVG_F, fall back to TAVG_C or TMAX
temp_col = None
for c in ["TAVG_F", "TAVG_C", "TMAX"]:
    if c in daily.columns:
        temp_col = c
        break

fig_dual = make_subplots(specs=[[{"secondary_y": True}]])
# temperature on left (y1), trips on right (y2) or vice-versa—your choice
fig_dual.add_trace(
    go.Scatter(x=daily["date"], y=daily[temp_col], name=f"Temperature ({temp_col})", mode="lines"),
    secondary_y=False
)
fig_dual.add_trace(
    go.Scatter(x=daily["date"], y=daily["trip_count"], name="Trips", mode="lines"),
    secondary_y=True
)
fig_dual.update_layout(
    title="Daily Trips vs Temperature (LaGuardia)",
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="left", x=0)
)
fig_dual.update_yaxes(title_text=f"Temperature ({temp_col})", secondary_y=False)
fig_dual.update_yaxes(title_text="Trips", secondary_y=True)
fig_dual.show()

# (Optional) Save HTMLs for quick share/check
fig_bar.write_html("bar_top_stations.html")
fig_dual.write_html("dual_trips_temp.html")

print("Charts rendered. Add nyc_trip_map.html to your Streamlit app folder to embed the Kepler map.")

✅ Saved top_stations.csv
Saved daily_trips.csv (with weather columns)


Charts rendered. Add nyc_trip_map.html to your Streamlit app folder to embed the Kepler map.


In [55]:
# app2.py – Streamlit Dashboard

import streamlit as st
import streamlit.components.v1 as components
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
from pathlib import Path
import gzip

# ---- Page Config ----
st.set_page_config(page_title="NYC Citi Bike Dashboard", layout="wide")

st.title("NYC Citi Bike – Interactive Dashboard")
st.write("Explore station popularity, seasonality vs temperature, and flows (Kepler map).")

# ---- Cached Data Loader ----
@st.cache_data
def load_data():
    daily = pd.read_csv("daily_trips.csv", parse_dates=["date"])
    top_stations = pd.read_csv("top_stations.csv")
    return daily, top_stations

# ---- File Guard ----
missing = [p for p in ["daily_trips.csv", "top_stations.csv"] if not os.path.exists(p)]
if missing:
    st.error(f"Missing files: {', '.join(missing)}. Generate them from your notebook first.")
    st.stop()

daily, top_stations = load_data()

# ---- Sidebar Filters ----
with st.sidebar:
    st.header("Filters")
    dmin, dmax = daily["date"].min().date(), daily["date"].max().date()
    date_sel = st.date_input("Date range", (dmin, dmax), min_value=dmin, max_value=dmax)

    if isinstance(date_sel, tuple) and len(date_sel) == 2:
        start, end = date_sel
    else:
        start, end = dmin, dmax

    daily_f = daily[(daily["date"] >= pd.to_datetime(start)) & (daily["date"] <= pd.to_datetime(end))]

# ---- Plotly Bar: Top Stations ----
fig_bar = px.bar(
    top_stations.head(20),
    x="start_station_name",
    y="trip_count",
    title="Top 20 Most Popular Start Stations",
    color="trip_count",
    color_continuous_scale="Blues",
    labels={"start_station_name": "Station", "trip_count": "Trips"}
)
fig_bar.update_layout(xaxis_tickangle=45)

st.subheader("Most Popular Stations")
st.plotly_chart(fig_bar, use_container_width=True)

# ---- Plotly Dual-axis: Trips vs Temperature ----
temp_col = next((c for c in ["TAVG_F", "TAVG_C", "TMAX"] if c in daily_f.columns), None)

if temp_col is None:
    st.warning("No temperature column found (expected TAVG_F, TAVG_C, or TMAX). Showing trips only.")
    fig_dual = go.Figure(go.Scatter(x=daily_f["date"], y=daily_f["trip_count"], name="Trips", mode="lines"))
else:
    fig_dual = make_subplots(specs=[[{"secondary_y": True}]])
    fig_dual.add_trace(
        go.Scatter(x=daily_f["date"], y=daily_f[temp_col], name=f"Temperature ({temp_col})", mode="lines"),
        secondary_y=False
    )
    fig_dual.add_trace(
        go.Scatter(x=daily_f["date"], y=daily_f["trip_count"], name="Trips", mode="lines"),
        secondary_y=True
    )
    fig_dual.update_yaxes(title_text=f"Temperature ({temp_col})", secondary_y=False)
    fig_dual.update_yaxes(title_text="Trips", secondary_y=True)

fig_dual.update_layout(title="Daily Trips vs Temperature", legend=dict(orientation="h", y=1.05))

st.subheader("Trips vs Temperature")
st.plotly_chart(fig_dual, use_container_width=True)

# ---- Kepler Map (embed the exported HTML) ----
st.subheader("Flows (Kepler.gl Map)")

def load_map_html():
    gz = Path("nyc_trip_map.html.gz")
    if gz.exists():
        with gzip.open(gz, "rt", encoding="utf-8", errors="ignore") as f:
            return f.read()
    html = Path("nyc_trip_map.html")
    if html.exists():
        return html.read_text(encoding="utf-8", errors="ignore")
    return None

map_html = load_map_html()

if map_html:
    components.html(map_html, height=620, scrolling=True)
else:
    st.warning("Add nyc_trip_map.html or nyc_trip_map.html.gz to this folder to display the Kepler map.")

2025-08-21 19:25:55.068 No runtime found, using MemoryCacheStorageManager
